# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [48]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import BaseEstimator, CountVectorizer, TfidfTransformer, TransformerMixin
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import FeatureUnion


import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///database.db')
df = pd.read_sql("select * from df_db1", con = engine)

In [3]:
df.shape

(26028, 40)

In [4]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [5]:
df['genre'].nunique()

3

In [6]:
df.groupby('genre').count()['id']

genre
direct    10634
news      13036
social     2358
Name: id, dtype: int64

In [7]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [8]:
X.head(2)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

In [9]:
Y.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [10]:
X.shape

(26028,)

In [11]:
Y.shape

(26028, 36)

In [12]:
type(X)

pandas.core.series.Series

In [13]:
type(Y)

pandas.core.frame.DataFrame

In [14]:
X.shape

(26028,)

In [15]:
len(X.unique())

25990

In [16]:
Y.shape

(26028, 36)

In [17]:
Y.drop_duplicates().shape

(4002, 36)

In [18]:
# Now, X which is the list of messages, has some duplicates (26028 versus 25990). But, since the Y values, which are the 1/0 
# labels take very few unique values, we will not be able to drop any X values even if there are some duplicate messages.
# We will use all data to train

### 2. Write a tokenization function to process your text data

In [19]:
def tokenize(text):
    a = word_tokenize(text.lower().strip())
    b = [i for i in a if i not in stopwords.words("english")]
    
    c = []
    for i in b:
        c.append(WordNetLemmatizer().lemmatize(i))
        
    return c

In [20]:
a = 'The Medan Chapter of the Taiwan Buddhist Tzu C.'

In [21]:
tokenize(a)

['medan', 'chapter', 'taiwan', 'buddhist', 'tzu', 'c', '.']

In [22]:
word_tokenize(a)

['The', 'Medan', 'Chapter', 'of', 'the', 'Taiwan', 'Buddhist', 'Tzu', 'C', '.']

In [23]:
from nltk.tokenize import sent_tokenize

In [24]:
sent_tokenize(a)

['The Medan Chapter of the Taiwan Buddhist Tzu C.']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [112]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

In [113]:
Y.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [114]:
Y.shape

(26028, 36)

In [30]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

In [29]:
from sklearn.pipeline import Pipeline

In [117]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(estimator=RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)

In [119]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [25]:
from sklearn.metrics import classification_report

In [121]:
pred = pipeline.predict(X_test)
# pred = pipeline.predict(X_train)

In [122]:
pred.shape

(8590, 36)

In [123]:
X_test.shape

(8590,)

In [124]:
y_test.shape

(8590, 36)

In [125]:
y_test.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12293,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
25482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
pred

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [127]:
pred1 = pd.DataFrame(pred)

In [128]:
pred1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [129]:
y_test.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
20861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12293,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [130]:
pred1.shape

(8590, 36)

In [131]:
y_test.shape

(8590, 36)

In [132]:
len(pred1)

8590

In [133]:
pred1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [136]:
print(classification_report(y_test.values, pred, target_names = y_test.columns))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      6547
               request       0.82      0.42      0.55      1464
                 offer       0.00      0.00      0.00        37
           aid_related       0.75      0.60      0.67      3615
          medical_help       0.57      0.07      0.13       701
      medical_products       0.73      0.12      0.21       441
     search_and_rescue       0.77      0.04      0.07       264
              security       0.00      0.00      0.00       167
              military       0.57      0.08      0.14       289
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.40      0.54       564
                  food       0.84      0.45      0.59       968
               shelter       0.79      0.26      0.40       780
              clothing       0.61      0.09      0.15       129
                 money       0.67      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
# So, in this above model, the total precision is 0.74 and f-1 score is .53
# We will try to improve these values in the next step

### 6. Improve your model
Use grid search to find better parameters. 

In [137]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(estimator=RandomForestClassifier()))
])

In [138]:
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])>

In [170]:
parameters = {'vect__max_df': [0.5, 0.75], 
#              'vect__ngram_range': [1, 5],
#              'tfidf__sublinear_tf': [True, False],
#              'tfidf__smooth_idf': [True, False],
#               'tfidf__norm': ['l1', 'l2', None]
            
             'model__estimator__max_depth': [2, 3],
             'model__estimator__n_estimators': [5, 10],
             }

cv = GridSearchCV(pipeline, parameters)

In [171]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': [0.5, 0.75], 'model__estimator__max_depth': [2, 3], 'model__estimator__n_estimators': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [172]:
pd.set_option('display.max_colwidth', -1)

In [178]:
pd.DataFrame(cv.cv_results_)[['params', 'mean_test_score']].sort_values(by = 'mean_test_score', ascending = False)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

,params,mean_test_score
2,"{'model__estimator__max_depth': 2, 'model__estimator__n_estimators': 10, 'vect__max_df': 0.5}",0.200310
1,"{'model__estimator__max_depth': 2, 'model__estimator__n_estimators': 5, 'vect__max_df': 0.75}",0.200080
3,"{'model__estimator__max_depth': 2, 'model__estimator__n_estimators': 10, 'vect__max_df': 0.75}",0.200080
7,"{'model__estimator__max_depth': 3, 'model__estimator__n_estimators': 10, 'vect__max_df': 0.75}",0.200023
6,"{'model__estimator__max_depth': 3, 'model__estimator__n_estimators': 10, 'vect__max_df': 0.5}",0.199736
0,"{'model__estimator__max_depth': 2, 'model__estimator__n_estimators': 5, 'vect__max_df': 0.5}",0.198704
4,"{'model__estimator__max_depth': 3, 'model__estimator__n_estimators': 5, 'vect__max_df': 0.5}",0.198245
5,"{'model__estimator__max_depth': 3, 'model__estimator__n_estimators': 5, 'vect__max_df': 0.75}",0.197729


In [174]:
pred_tuned = cv.predict(X_test)

In [175]:
pred_tuned.shape

(8590, 36)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [179]:
print(classification_report(y_test.values, pred_tuned, target_names = y_test.columns))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.87      6547
               request       0.00      0.00      0.00      1464
                 offer       0.00      0.00      0.00        37
           aid_related       0.91      0.02      0.04      3615
          medical_help       0.00      0.00      0.00       701
      medical_products       0.00      0.00      0.00       441
     search_and_rescue       0.00      0.00      0.00       264
              security       0.00      0.00      0.00       167
              military       0.00      0.00      0.00       289
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       564
                  food       0.00      0.00      0.00       968
               shelter       0.00      0.00      0.00       780
              clothing       0.00      0.00      0.00       129
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [202]:
# Clearly, with these set of hyper parameters, the performance of the model has become worse 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [203]:
# Let's user AdaBoostClassifier to improve the model's performance

In [56]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [57]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(estimator = AdaBoostClassifier()))
])

In [58]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [60]:
print(classification_report(y_test.values, pipeline.predict(X_test), target_names = y_test.columns))

                        precision    recall  f1-score   support

               related       0.80      0.95      0.87      6547
               request       0.76      0.53      0.62      1464
                 offer       0.14      0.03      0.05        37
           aid_related       0.75      0.61      0.67      3615
          medical_help       0.58      0.24      0.34       701
      medical_products       0.63      0.31      0.42       441
     search_and_rescue       0.62      0.19      0.29       264
              security       0.30      0.04      0.06       167
              military       0.63      0.30      0.41       289
           child_alone       0.00      0.00      0.00         0
                 water       0.75      0.65      0.70       564
                  food       0.79      0.68      0.73       968
               shelter       0.79      0.54      0.64       780
              clothing       0.67      0.43      0.53       129
                 money       0.54      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [204]:
# We will keep f-1 score as the metric to judge finally. Because for unbalanced dataset such as this one, f-1 score which
# is a harmonin mean between "Predicion" and "Recall" is apprpriate without biasing neither towards
# False positives or False negatives
# f-1 score = 2*Precision*Recall/(Precision + Recall)

In [ ]:
# So, there with AdaBoost, the f-1 score has increased from 53% to 63%

In [196]:
# Using Randomized Search CV with AdaBoost.
# We will increase the number of parameters in the Grid

In [197]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('model', MultiOutputClassifier(estimator = AdaBoostClassifier()))
])

In [198]:
parameters = {'vect__max_df': (0.1, 0.2, 0.5, 0.75, 1),
              'vect__max_features': (None, 5000, 10000),
             'vect__ngram_range': ((1, 1), (1, 2)),
             'tfidf__use_idf': [True, False],
#              'tfidf__smooth_idf': [True, False],
#               'tfidf__norm': ['l1', 'l2', None]
            
#              'model__estimator__max_depth': [2, 3, 4, 5, 6],
             'model__estimator__n_estimators': [5, 10, 50, 100, 200],
             'model__estimator__learning_rate': [0.01, 0.05]
#              'model__estimator__subsample': [0.4, 0.5, 0.6]  
             }

In [199]:
cv = RandomizedSearchCV(pipeline, parameters, cv = 5, n_iter = 10, n_jobs = 4, verbose = True)

In [200]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 310.8min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed: 347.8min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
          fit_params=None, iid=True, n_iter=10, n_jobs=4,
          param_distributions={'vect__max_df': (0.1, 0.2, 0.5, 0.75, 1), 'vect__max_features': (None, 5000, 10000), 'vect__ngram_range': ((1, 1), (1, 2)), 'tfidf__use_idf': [True, False], 'model__estimator__n_estimators': [5, 10, 50, 100, 200], 'model__estimator__learning_rate': [0.01, 0.05]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=True)

In [205]:
print(classification_report(y_test.values, cv.predict(X_test), target_names = y_test.columns))

                        precision    recall  f1-score   support

               related       0.77      1.00      0.87      6547
               request       0.82      0.34      0.48      1464
                 offer       0.00      0.00      0.00        37
           aid_related       0.78      0.44      0.56      3615
          medical_help       0.63      0.07      0.13       701
      medical_products       0.68      0.10      0.18       441
     search_and_rescue       0.88      0.06      0.11       264
              security       0.00      0.00      0.00       167
              military       0.59      0.04      0.08       289
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.55      0.66       564
                  food       0.76      0.71      0.73       968
               shelter       0.84      0.42      0.56       780
              clothing       0.79      0.24      0.37       129
                 money       0.68      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [206]:
pd.DataFrame(cv.cv_results_)[['params', 'mean_test_score']].sort_values(by = 'mean_test_score', ascending = False)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

,params,mean_test_score
0,"{'vect__ngram_range': (1, 1), 'vect__max_features': None, 'vect__max_df': 0.2, 'tfidf__use_idf': False, 'model__estimator__n_estimators': 200, 'model__estimator__learning_rate': 0.05}",0.221872
7,"{'vect__ngram_range': (1, 1), 'vect__max_features': 5000, 'vect__max_df': 0.75, 'tfidf__use_idf': False, 'model__estimator__n_estimators': 200, 'model__estimator__learning_rate': 0.05}",0.221642
2,"{'vect__ngram_range': (1, 1), 'vect__max_features': 10000, 'vect__max_df': 0.5, 'tfidf__use_idf': True, 'model__estimator__n_estimators': 200, 'model__estimator__learning_rate': 0.05}",0.221241
4,"{'vect__ngram_range': (1, 1), 'vect__max_features': None, 'vect__max_df': 0.1, 'tfidf__use_idf': False, 'model__estimator__n_estimators': 200, 'model__estimator__learning_rate': 0.05}",0.220954
8,"{'vect__ngram_range': (1, 2), 'vect__max_features': 5000, 'vect__max_df': 0.5, 'tfidf__use_idf': True, 'model__estimator__n_estimators': 200, 'model__estimator__learning_rate': 0.01}",0.209428
5,"{'vect__ngram_range': (1, 1), 'vect__max_features': None, 'vect__max_df': 0.75, 'tfidf__use_idf': False, 'model__estimator__n_estimators': 200, 'model__estimator__learning_rate': 0.01}",0.207994
6,"{'vect__ngram_range': (1, 2), 'vect__max_features': None, 'vect__max_df': 1, 'tfidf__use_idf': True, 'model__estimator__n_estimators': 10, 'model__estimator__learning_rate': 0.05}",0.199449
3,"{'vect__ngram_range': (1, 2), 'vect__max_features': 5000, 'vect__max_df': 0.75, 'tfidf__use_idf': False, 'model__estimator__n_estimators': 10, 'model__estimator__learning_rate': 0.05}",0.194518
1,"{'vect__ngram_range': (1, 2), 'vect__max_features': 5000, 'vect__max_df': 0.75, 'tfidf__use_idf': False, 'model__estimator__n_estimators': 10, 'model__estimator__learning_rate': 0.01}",0.182819
9,"{'vect__ngram_range': (1, 1), 'vect__max_features': None, 'vect__max_df': 0.2, 'tfidf__use_idf': False, 'model__estimator__n_estimators': 5, 'model__estimator__learning_rate': 0.01}",0.179608


In [59]:
# So, the best model is the one with no change in parameters from the default one which has the best f1-score of 63%
# We will finalize this model

### 9. Export your model as a pickle file

In [67]:
pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [54]:
import pickle

In [61]:
filename = 'finalized_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.